In [3]:
import random
import pygame
import numpy as np
from IPython.display import display

random_seed = 42

# 1. Encoding

We will encode N-queens problem as a set of queens positions in the matrix of size $N \times N$, filled with $N$ queens on random positions.

In [5]:
def create_initiate_state(n: int):
    """
    Function to generate a matrix of size n x n with n queens at random positions
    """
    all_possible_positions = [(i, j) for i in range(n) for j in range(n)]
    sample = random.sample(all_possible_positions, n)
    return set(sample)


In [6]:
random.seed(random_seed)
initial_state = create_initiate_state(5)
display(initial_state)

{(0, 0), (0, 3), (1, 3), (4, 0), (4, 3)}

So, our state is defined as a set of positions - for each of $N$ queens.

Let's visualize queens positions using a matrix, where value $1$ represents queen on that position and $0$ represents empty tile.

Also, note that we use coordinate system where $(0,0)$ is in the left top corner.

In [7]:
def get_matrix_view(positions: set, n: int):
    matrix = np.zeros((n, n), dtype=int)
    for i, j in positions:
        matrix[i][j] = 1
    return matrix.T

In [8]:
matrix_view = get_matrix_view(initial_state, 5)
display(matrix_view)

array([[1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1],
       [0, 0, 0, 0, 0]])

![](images/example1.png)

# 2. Objective Function

As an objective function we will use the number of pairs of queens that are attacking each other (it counts as an attack if two pieces are in the same line, even if there is an intervening piece between them). Pairs are not ordered, so $(Q_1,Q_2) = (Q_2, Q_1)$.

For this we need to implement a function, which will get all possible tiles where each queen can move, and then check how many queens are on those positions. After that we need to exclude the queen from the set of queens to avoid duplications.

In the following function, which will compute all tiles where a queen can go, there is a parameter _obstacles_. This parameter will be ignored for the heuristic function, but will be used later for computing states - because we cannot move our queen on and through another queen when we consider other states.

Also, this function has a _depth_ parameter, which represents how far we allow our queen to go. We can use this to limit number of queen moves. 

In [9]:
def get_queen_moves(queen: tuple[int, int], n: int, depth: int, obstacles: set | None = None):
    neighbors = set()
    q_row, q_col = queen

    horizontal_directions = [(0, 1), (0, -1)]
    vertical_directions = [(1, 0), (-1, 0)]
    diag_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    directions = horizontal_directions + vertical_directions + diag_directions

    # for each direction find all positions where the queen can move
    for direction in directions:
        d_row, d_col = direction
        c_row, c_col = q_row + d_row, q_col + d_col
        current_depth = 1
        while 0 <= c_row < n and 0 <= c_col < n and current_depth <= depth:
            if obstacles and (c_row, c_col) in obstacles:
                break
            neighbors.add((c_row, c_col))
            c_row, c_col = c_row + d_row, c_col + d_col
            current_depth += 1
    return neighbors

Now, the heuristic function will compute amount of collisions - for each queen it will get all its moves, then remove the queen from queens set and count a number of collisions - number of attacked other queens. We remove the queen from the set because we want to avoid duplicate counts.

In [10]:
def heuristic(queens: set):
    h = 0
    n = len(queens)
    remaining_queens: set = queens.copy()
    while len(remaining_queens) > 0:
        queen = remaining_queens.pop()
        neighbors = get_queen_moves(queen, n, depth=n)
        h += len(neighbors & remaining_queens)
    return h


In [11]:
heuristic(initial_state)

7

For reference:

![](images/example1.png)

# Neighbor States Definition

As a neighbor state we will count each unique set of queens position after one of the queen moved to one of its possible move positions. A queen cannot move on the tile where another queen is, and a queen cannot move through another queen. 

If we use the _depth_ parameter, the situation is different - a queen cannot move farther that the depth distance. If depth is $1$, then a queen can move only on adjacent tiles (if there are no other queens). The _depth_ parameter gives us different neighbor states definitions.

# Enumeration of States

For enumeration we will just count the number of all unique states:


In [197]:
def get_all_neighbor_states(queens: set, depth: int):
    """
    Gets all possible distinct states for each queen, returned as a list of states.
    """
    n = len(queens)
    neighbors_states = list()
    for queen in queens:
        remaining_queens = queens - {queen}
        neighbors = get_queen_moves(queen, n, depth, remaining_queens)
        for neighbor in neighbors:
            neighbors_states.append(remaining_queens | {neighbor})
    return neighbors_states

In [198]:
def enumerate_neighbor_states(queens: set, depth: int):
    all_neighbor_states = get_all_neighbor_states(queens, depth)
    all_neighbor_states_count = len(all_neighbor_states)
    print(f"Number of neighbor states for depth {depth}: {all_neighbor_states_count}")


In [200]:
enumerate_neighbor_states(initial_state, 1)
enumerate_neighbor_states(initial_state, 2)
enumerate_neighbor_states(initial_state, 3)
enumerate_neighbor_states(initial_state, 4)
enumerate_neighbor_states(initial_state, 5)

Number of neighbor states for depth 1: 22
Number of neighbor states for depth 2: 36
Number of neighbor states for depth 3: 42
Number of neighbor states for depth 4: 43
Number of neighbor states for depth 5: 43


For reference:

![](images/example1.png)

# Hill Climbing

Now let's try to solve the N-queen problem for any N.

We will use **random-restart hill climbing** (thus the _restarts_ parameter) with **sideways moves**.

In [201]:
def get_best_state(states: list):
    """
    Function will get all states, iterate through each state and find the state with minimal heuristic cost and return it.
    """
    best_state = states.pop()
    best_h = heuristic(best_state)
    for state in states:
        h = heuristic(state)
        if h < best_h:
            best_h = h
            best_state = state
    return best_state, best_h

In [202]:
def hill_climbing(initial_state, depth: int, side_way_moves: int = 0, ):
    current_state, current_h = initial_state, heuristic(initial_state)
    current_side_way_moves = 0
    while True:
        all_neighbors = get_all_neighbor_states(current_state, depth)
        best_neighbor_state, best_neighbor_h = get_best_state(all_neighbors)
        if best_neighbor_h == current_h:
            if current_side_way_moves < side_way_moves:
                current_side_way_moves += 1
            else:
                return current_state, current_h
        elif best_neighbor_h > current_h:
            return current_state, current_h
        current_state, current_h = best_neighbor_state, best_neighbor_h


In [203]:
def n_queens(n: int, restarts: int = 0, side_way_moves: int = 0, depth: int | None = None, rd_seed: int | None = None):
    if depth is None:
        depth = n
    elif depth < 1:
        raise ValueError("depth must be at least 1")
    if rd_seed:
        random.seed(rd_seed)
    else:
        random.seed()

    initial_state = create_initiate_state(n)
    best_state, best_h = hill_climbing(initial_state, depth, side_way_moves)
    if best_h == 0:
        return best_state, best_h
    for restart in range(restarts):
        initial_state = create_initiate_state(n)
        current_best_state, current_best_h = hill_climbing(initial_state, depth, side_way_moves)
        if current_best_h < best_h:
            best_state, best_h = current_best_state, current_best_h
        if current_best_h == 0:
            break
    return best_state, best_h

Now lets try it for different parameters:

In [211]:
n = 8
restarts = 1
side_way_moves = 5
depth = None
rd_seed = None

best_state, best_h = n_queens(n, restarts, side_way_moves, depth, rd_seed)

print(f"{n}-queens problem with best heuristic: {best_h}")
display(get_matrix_view(best_state, n))

8-queens problem with best heuristic: 0


array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0]])

# Visualization

Now lets visualize our algorithm with pygame:

In [126]:
TILE_SIZE = 40
FONT_SIZE = 20
FONT_COLOR = (0, 0, 0)
BLACK_TILE_COLOR = (0, 0, 0)
WHITE_TILE_COLOR = (255, 255, 255)
QUEEN_COLOR = (176, 65, 65)
CIRCLE_RADIUS = TILE_SIZE / 2 - 2

In [127]:
def draw_tile(surface, position, color, circle=False, offset=(0, 0)):
    x_pos, y_pos = position
    x_offset, y_offset = offset
    x, y = x_pos + x_offset, y_pos + y_offset

    if circle:
        pygame.draw.circle(surface=surface, color=color,
                           center=(x * TILE_SIZE + TILE_SIZE / 2, y * TILE_SIZE + TILE_SIZE / 2), radius=CIRCLE_RADIUS)
    else:
        pygame.draw.rect(surface=surface, color=color,
                         rect=(x * TILE_SIZE, y * TILE_SIZE, TILE_SIZE, TILE_SIZE))


def draw_tiles(surface, positions, color, circle=False, display=True, offset=(0, 0)):
    for position in positions:
        draw_tile(surface, position, color, circle, offset)
    if display:
        pygame.display.flip()


def draw_text(position, font, screen, text, color):
    img_expanded = font.render(text, True, color)
    screen.blit(img_expanded, position)


def draw_chessboard(screen, n, offset=(0, 0), display=True):
    x_offset, y_offset = offset
    for x in range(n):
        for y in range(n):
            color = WHITE_TILE_COLOR
            if (x + y) % 2 == 0:
                color = BLACK_TILE_COLOR
            draw_tile(screen, (x + x_offset, y + y_offset), color)
    if display:
        pygame.display.flip()

In [144]:
def run_visualization(n: int, side_way_moves: int = 0, depth: int | None = None,
                      rd_seed: int | None = None):
    if depth is None:
        depth = n
    elif depth < 1:
        raise ValueError("depth must be at least 1")

    if rd_seed:
        random.seed(rd_seed)
    else:
        random.seed()

    pygame.init()
    screen = pygame.display.set_mode((2 * n * TILE_SIZE + TILE_SIZE, n * TILE_SIZE + 4 * FONT_SIZE))
    font = pygame.font.Font("roboto.ttf", FONT_SIZE)

    screen.fill("gray")
    pygame.display.flip()

    initial_state = create_initiate_state(n)
    initial_h = heuristic(initial_state)

    draw_chessboard(screen, n, display=False)
    draw_text(position=(TILE_SIZE / 2, n * TILE_SIZE + FONT_SIZE / 2), font=font, screen=screen, text="INITIAL",
              color=FONT_COLOR)
    draw_text(position=(TILE_SIZE / 2, n * TILE_SIZE + 2 * FONT_SIZE), font=font, screen=screen, text=f"h: {initial_h}",
              color=FONT_COLOR
              )
    draw_tiles(surface=screen, positions=initial_state, color=QUEEN_COLOR, circle=True, display=False)
    pygame.display.flip()

    best_state, best_h = hill_climbing(initial_state, depth, side_way_moves)

    draw_chessboard(screen, n, (n + 1, 0))
    draw_text(position=(TILE_SIZE * n + 1.5 * TILE_SIZE, n * TILE_SIZE + FONT_SIZE / 2), font=font, screen=screen,
              text="FINAL",
              color=FONT_COLOR)
    draw_text(position=(TILE_SIZE * n + 1.5 * TILE_SIZE, n * TILE_SIZE + 2 * FONT_SIZE), font=font, screen=screen,
              text=f"h: {best_h}",
              color=FONT_COLOR)
    draw_tiles(surface=screen, positions=best_state, color=QUEEN_COLOR, circle=True, display=False, offset=(n + 1, 0))
    pygame.display.flip()

    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
    pygame.quit()

In [215]:
n = 8
side_way_moves = 5
depth = None
rd_seed = 43

run_visualization(n, side_way_moves, depth, rd_seed)

n = 8, side_way_moves = 5, depth = None, rd_seed = 42

![](images/example2.png)

n = 8, side_way_moves = 5, depth = None, rd_seed = 43

![](images/example3.png)